# TP 4 RNNs: classification de séquences, forecasting, génération de séquences, one-hot encoding

_Ismaël Bonneau_

Ce notebook sert uniquement à présenter nos résultats, et les bouts de code intéressants dans le cadre de ce rapport. L'intégralité du code est contenu dans le fichier tp4.py.

## But

- implémenter un RNN en pytorch
- réaliser une tâche de classification de séquence
- réaliser une tâche de forecasting (prédiction de caracètre)

## Données

2 jeux de données sont fournis: un jeu de relevés de température à travers 31 villes des Etats Unis et du Canada, qui pourra servir à de la classification de séquence (many to one), par exemple pour prédire une ville sachant une séquence de température, ou à du forecasting, en préduisant la température à ${t+1}$. L'autre est un jeu de données de discours de trump, qui pourra servir essentiellement à du forecasting.

In [1]:
import numpy as np
import torch
import torch.nn as nn

import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

import models
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
temperatures_csv = pd.read_csv("data/tempAMAL_train.csv")
print("Nb exemples: {}, cities: {}".format(temperatures_csv.shape[0], temperatures_csv.shape[1]))

temperatures_csv.head(5)

Nb exemples: 11115, cities: 31


,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Detroit,Jacksonville,Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New York,Montreal,Boston
0,2012-10-01 13:00:00,284.630000,282.080000,289.480000,281.800000,291.870000,291.530000,293.410000,296.600000,285.120000,...,284.030000,298.170000,288.650000,299.720000,281.000000,286.260000,285.630000,288.220000,285.830000,287.170000
1,2012-10-01 14:00:00,284.629041,282.083252,289.474993,281.797217,291.868186,291.533501,293.403141,296.608509,285.154558,...,284.069789,298.205230,288.650172,299.732518,281.024767,286.262541,285.663208,288.247676,285.834650,287.186092
2,2012-10-01 15:00:00,284.626998,282.091866,289.460618,281.789833,291.862844,291.543355,293.392177,296.631487,285.233952,...,284.173965,298.299595,288.650582,299.766579,281.088319,286.269518,285.756824,288.326940,285.847790,287.231672
3,2012-10-01 16:00:00,284.624955,282.100481,289.446243,281.782449,291.857503,291.553209,293.381213,296.654466,285.313345,...,284.278140,298.393961,288.650991,299.800641,281.151870,286.276496,285.850440,288.406203,285.860929,287.277251
4,2012-10-01 17:00:00,284.622911,282.109095,289.431869,281.775065,291.852162,291.563063,293.370249,296.677445,285.392738,...,284.382316,298.488326,288.651401,299.834703,281.215421,286.283473,285.944057,288.485467,285.874069,287.322831


### Construisons le jeu de données

On va construire un dataset et un dataLoader associé qui vont contenir nos séquences et batches associés. On procède comme suit:

- On sample au hasard 10 villes


In [23]:
nbclass = 10 # prendre nbclass = 10 villes pour la tache de classif de séquences
seq_len = 20 # prendre une longueur de séquence de taille fixe pour le moment
nb_sequences = 300 # prendre 300 séquences par ville

cities = np.random.choice(temperatures_csv.columns[1:], nbclass, replace=False) # ne pas sélectionner le datetime

def sample_sequence(city):
    start = np.random.randint(0, temperatures_csv.shape[0] - seq_len)
    return np.array(temperatures_csv[city][start:start + seq_len])

sequences = []
labels = []
for i, city in enumerate(cities):
    j = 0
    while j < nb_sequences:
        s = sample_sequence(city)
        if not np.isnan(s).any(): # certains nan peuvent être présents dans le dataframe
            sequences.append(s)
            labels.append(i)
            j += 1
        
sequences = np.array(sequences)
labels = np.array(labels)
sequences /= np.max(sequences) # normaliser
#sequences, labels = shuffle(sequences, labels, random_state=1997) # shuffle lines
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=1997)

X_train = np.expand_dims(X_train, axis=2) # pour créer une dimension supplémentaire de taille 1
X_test = np.expand_dims(X_test, axis=2)

assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]
print(X_train.shape, X_test.shape)

(2400, 20, 1) (600, 20, 1)


In [4]:
cities

array(['Chicago', 'Albuquerque', 'Pittsburgh', 'Miami', 'Denver',
       'Minneapolis', 'Saint Louis', 'Indianapolis', 'Las Vegas',
       'San Diego'], dtype=object)

Le RNN que nous avons codé attend en entrée des matrices de taille sequence_length x batch x dim. Dans notre cas où nous avons pris des séquences de taille 20, on aurait donc des matrices de taille 20 x batch x 1, car nos données sont en dimension 1.

On va donc swaper les dimensions 0 et 1 de notre array de séquences:

In [24]:
print(X_train.shape)
X_train = np.swapaxes(X_train,0,1)
X_test = np.swapaxes(X_test,0,1)
print(X_train.shape)

(2400, 20, 1)
(20, 2400, 1)


### Un petit dataset pour le RNN s'impose:

In [47]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class Dataset_RNN(Dataset):
    """Dataset avec la taille de batch en axis 1 au lieu de 0.
        Pas mal pour notre RNN"""
    def __init__(self, x, y):
        super(Dataset_RNN, self).__init__()
        self.labels = torch.from_numpy(y)
        self.data = torch.from_numpy(x).float()
    def __getitem__(self, index):
        return self.data[:,index,:], self.labels[index]
    def __len__(self):
        return len(self.labels)

## Modèles

### Le code du RNN: 

In [7]:
class RNN(torch.nn.Module):
    """docstring for RNN"""
    def __init__(self, dim, latent):
        super(RNN, self).__init__()
        self.dim = dim
        self.latent = latent
        self.Wx = torch.nn.Linear(dim, latent)
        self.Wh = torch.nn.Linear(latent, latent)

    def forward(self, x, h=None):
        """ x: sequence_length x batch x dim 
            h: batch x latent
            returns: length x batch x latent Tensor"""
        historique = []
        # pour chaque instant de la sequence:
        if h is None:
            ht = torch.zeros(x.size()[1], self.latent)
            historique.append(ht.numpy())
        for i, xt in enumerate(x):
            # ht: (batch x latent)
            ht = self.one_step(xt, ht)
            if i > 0:
                historique.append(ht) # Ne pas enregistrer les h0

        return torch.Tensor(historique)

    def one_step(self, x, h):
        """ x: batch x dim 
            h: batch x latent
            returns: batch x latent Tensor """
        return torch.nn.functional.leaky_relu(torch.add(self.Wx(x), self.Wh(h)))

### Le code du decoder qui va permettre de passer de ${h_{T}}$ à une classe

In [73]:
class Decoder(torch.nn.Module):
    """ simple Neural Net """
    def __init__(self, inSize, outSize, layers=[]):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList([])
        for x in layers:
            self.layers.append(nn.Linear(inSize, x))
            inSize = x
        self.layers.append(nn.Linear(inSize, outSize))

    def forward(self, x):
        x = self.layers[0](x)
        for i in range(1, len(self.layers)):
            x = torch.nn.functional.leaky_relu(x)
            x = self.layers[i](x)
        return torch.nn.functional.softmax(x)

### SequenceClassifier: contient un RNN et un decoder.

In [74]:
class SequenceClassifier(torch.nn.Module):
    """ """
    def __init__(self, dim, latent, nbClass):
        super(SequenceClassifier, self).__init__()
        self.rnn = RNN(dim, latent)
        self.decoder = Decoder(latent, nbClass, layers=[8, 16, 8])

    def forward(self, x):
        """ """
        hT = self.rnn(x)[-1]
        return self.decoder(hT)

## Expérimentations

Nous allons expérimenter avec le réseau de neurones.

- Criterion: Cross entropy loss
- Validation mesure: accuracy (classes équilibrées)

In [75]:
train_dataset = Dataset_RNN(X_train, y_train)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=240, shuffle=True)
test_dataset = Dataset_RNN(X_test, y_test)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=240, shuffle=False)

In [78]:
from torch.utils.tensorboard import SummaryWriter

def accuracy(model, x, labels):
    outputs = model(x)
    _, predicted = torch.max(outputs, 1)
    c = (predicted == labels).squeeze().sum()
    return c.item() / len(labels)

In [80]:
latentdim = 32
inputdim = 1 # on input des températures => scalaires

rnn = SequenceClassifier(inputdim, latentdim, len(cities))

criterion = nn.CrossEntropyLoss() # pas besoin de one-hot pour les labels avec cette fct là
optimizer = torch.optim.Adam(rnn.parameters(), lr=1e-3)

epochs = 10

writer = SummaryWriter()

for e in range(epochs):
    losses = []
    for seq_batch, label_batch in train_dataloader:
        optimizer.zero_grad()
        preds = rnn(seq_batch.permute(1, 0, 2))
        loss = criterion(preds, label_batch)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    accuracies_sum = 0
    i = 0
    with torch.no_grad():
        for seq_batch, label_batch in train_dataloader:
            rnn.eval()
            accuracies_sum += accuracy(rnn, seq_batch.permute(1, 0, 2), label_batch)
            i += 1
    print(accuracies_sum / i)
        
    #writer.add_scalar('Loss/train', epoch_losses[-1], e)
    print("epoch {} training loss {}".format(e, np.mean(losses)))
    
writer.close()

/home/ismael/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


0.10291666666666666
epoch 0 training loss 2.3030203104019167
0.10291666666666668
epoch 1 training loss 2.3030049324035646


KeyboardInterrupt: 

In [ ]:
PATH = './rnn.pth'
torch.save(rnn.state_dict(), PATH)